In [7]:
# Carles Payà Vidal

In [1]:
# librerias
import pandas as pd
import geopandas as gpd
import requests
from bs4 import BeautifulSoup
import schedule
import time
from datetime import datetime

In [2]:
# Funcion para descargar los datos de la EMT y extraer las próximas paradas de cada estación
# Ejecutar primero que el fichero de R hasta que se guarde el fichero emt_data_actual.csv y se siga ejecutando en segundo plano
def descargar_datos_emt():
    url = "https://valencia.opendatasoft.com/api/explore/v2.1/catalog/datasets/emt/exports/csv?lang=es&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
    dest_file = "datos/emt_data.csv"
    response = requests.get(url)
    with open(dest_file, 'wb') as f:
        f.write(response.content)
    emt_data = gpd.read_file(dest_file, encoding='utf-8', delimiter=';')
    
    emt_data['latitud'] = emt_data['geo_point_2d'].apply(lambda x: float(x.split(',')[0]))
    emt_data['longitud'] = emt_data['geo_point_2d'].apply(lambda x: float(x.split(',')[1]))
    emt_data = gpd.GeoDataFrame(emt_data, geometry=gpd.points_from_xy(emt_data.longitud, emt_data.latitud), crs=4326)
    
    emt_data = emt_data.rename(columns={'Pròximes Arribades / Proximas Llegadas':'Proximas_llegadas'})
    # Añadir columna 'hora_descarga' con la hora actual
    emt_data['hora_descarga'] = datetime.now().strftime('%H:%M:%S')
    
    def filtrar_informacion(row):
        enlace = row['Proximas_llegadas']
        response = requests.get(enlace)
        soup = BeautifulSoup(response.content, 'html.parser')
        informacion = [node.text for node in soup.find_all('div')]
        informacion_filtrada = [info for info in informacion if 'Próximo' in info or 'min.' in info]
        informacion_filtrada = [info for info in informacion_filtrada if 'Gestiona tu tiempo' not in info]
        informacion_filtrada = [info for info in informacion_filtrada if not info.split()[0].isdigit()]
        informacion_filtrada = ' '.join(informacion_filtrada).replace('\n', ' ')
        return informacion_filtrada

    emt_data['Proximas_llegadas'] = emt_data.apply(filtrar_informacion, axis=1)

    emt_data.to_csv('datos/emt_data_actual.csv', index=False)
    print("Archivo CSV guardado exitosamente")
    return emt_data

def job():
    datos_emt = descargar_datos_emt()
    # Realizar alguna operación con los datos descargados

schedule.every(3).minutes.do(job)

while True:
    schedule.run_pending()
    time.sleep(1)

Archivo CSV guardado exitosamente


KeyboardInterrupt: 